In [ ]:
from langchain.vectorstores import Weaviate
import weaviate
import os
from dotenv import load_dotenv
load_dotenv()

WEAVIATE_URL = os.getenv("WEAVIATE_URL")
WEAVIATE_API = os.getenv("WEAVIATE_API")

client = weaviate.Client(
    url=WEAVIATE_URL,
    auth_client_secret=weaviate.AuthApiKey(api_key=WEAVIATE_API))

/Users/prabeshsharma/Documents/QA_From_uploaded_pdf/env/lib/python3.12/site-packages/weaviate/__init__.py:144: DeprecationWarning: Dep010: Importing AuthApiKey from weaviate is deprecated. Import AuthApiKey from its module: weaviate.auth
  _Warnings.root_module_import(name, map_[name])
/var/folders/lv/cmxrpk6x201g31179k2jw81w0000gn/T/ipykernel_2344/2120763738.py:7: DeprecationWarning: 
Python client v3 `weaviate.Client(...)` connections and methods are deprecated and will
            be removed by 2024-11-30.

            Upgrade your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.
                - For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
                - For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration

            If you have to use v3 code, install the v3 client and pin the v3 dependency in your requirements file: `weaviate-client>=3.26.7;

In [2]:
# fixinf unicode error in google colab
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [4]:
# Specify embedding model (Using huggingface sentence transformer)
from langchain.embeddings import HuggingFaceEmbeddings
#model_kwargs = {"device":"cuda"}
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    #model_kwargs = model_kwargs
    )

/Users/prabeshsharma/Documents/QA_From_uploaded_pdf/env/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key = os.getenv("GROQ_API_KEY"),
    model="llama-3.1-70b-versatile",

)


In [7]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 20)

from langchain.prompts import ChatPromptTemplate

template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

prompt = ChatPromptTemplate.from_template(template)



In [ ]:
# def qafrompdf(pdf_directory,question):
#     loader = PyPDFLoader(pdf_directory,extract_images = True)
#     pages = loader.load()
    
#     docs = text_splitter.split_documents(pages)
#     vector_db = Weaviate.from_documents(
#     docs, embeddings, client = client, by_text=False)
#     output_parser=StrOutputParser()
#     retriever=vector_db.as_retriever()
#     rag_chain = (
#     {"context": retriever,  "question": RunnablePassthrough()}
#     | prompt
#     | llm
#     | output_parser
# )
#     ans = rag_chain.invoke(question)
#     return ans


In [20]:
import asyncio
from cachetools import TTLCache
import nest_asyncio


In [21]:
# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

# Initialize the cache (stores 100 answers for up to 10 minutes)
cache = TTLCache(maxsize=100, ttl=600)

async def process_pdf_async(pdf_directory):
    loader = PyPDFLoader(pdf_directory, extract_images=True)
    pages = loader.load()
    
    # Split documents efficiently
    docs = text_splitter.split_documents(pages)
    
    # Insert documents into the vector DB asynchronously
    vector_db = Weaviate.from_documents(
        docs, embeddings, client=client, by_text=False
    )
    
    return vector_db

async def qafrompdf_async(pdf_directory, question):
    # Check if the question is in cache
    if question in cache:
        return cache[question]

    # Process the PDF asynchronously
    vector_db = await process_pdf_async(pdf_directory)

    # Set up retrieval and RAG chain
    retriever = vector_db.as_retriever()
    output_parser = StrOutputParser()
    rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | output_parser
    )
    
    # Get the answer
    ans = rag_chain.invoke(question)
    
    # Store the answer in cache
    cache[question] = ans
    
    return ans

# Wrapper function for interactive environments
async def main(pdf_directory, question):
    return await qafrompdf_async(pdf_directory, question)

# Call this function in your interactive environment



In [22]:
pdf = "/Users/prabeshsharma/Downloads/Prabesh_24_oct_9_(Dynamic Routie Optimization research).pdf"
question = "what is Ant colony optimization"
result = await main(pdf, question)
print(result)

Ant Colony Optimization (ACO) is a bio-inspired optimization technique based on the foraging behavior of ants. It simulates the behavior of ants searching for optimal paths in a network to solve complex routing problems. ACO uses artificial "ants" as agents that move through the network, depositing pheromone on the paths they traverse. The intensity of the pheromone represents the quality or desirability of the path. Ants are attracted to paths with stronger pheromone concentrations, but they also explore new paths. The algorithm balances exploration and exploitation to find optimal solutions. ACO can be used for dynamic routing optimization, adapting to changing conditions such as traffic updates and new orders. It is a metaheuristic algorithm that can be used to find approximate solutions to complex problems.
